# First Project

## Section Three

### Problem no.1: Design a recommendation system

#### Adding Libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

#### Importing and checking the data

In [3]:
data = pd.read_csv('../Divar.csv')

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_7368\2330768946.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Divar.csv')


In [4]:
data.head()

,Unnamed: 0,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,user_type,description,title,rent_mode,...,property_type,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
0,0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,مشاور املاک,۵۰۰متر\n۲۰۰متر بنا دوبلکس\n۳خواب\nاستخر آبگرم ...,باغ ویلا اجاره روزانه استخر داخل لشکرآباد سهیلیه,NaN,...,NaN,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0
1,1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,مشاور املاک,دسترسی عالی به مترو و شریعتی \nمشاعات تمیز \nب...,۶۰ متر قلهک فول امکانات,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0
2,2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,NaN,تخلیه پایان ماه,آپارتمان ۳ خوابه ۱۳۲ متر,مقطوع,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN
3,3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,NaN,فرشته تاپ لوکیشن\n۹۰ متر موقعیت اداری\nیک اتاق...,فرشته ۹۰ متر دفتر کار مدرن موقعیت اداری,مقطوع,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,مشاور املاک,هلدینگ ساختمانی اکبری\n\nهمراه شما هستیم برای ...,۱۱۵ متری/شمالی رو به آفتاب/اکبری,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.shape

(1000000, 61)

#### Removing the useless Columns

In [6]:
data = data.set_index('Unnamed: 0')
data.index.name = None

In [7]:
data = data.drop(columns=['description', 'title'])
data2 = data.drop(columns=['user_type', 'rent_to_single'])

In [8]:
data2.head()

,cat2_slug,cat3_slug,city_slug,neighborhood_slug,created_at_month,rent_mode,rent_value,rent_type,price_mode,price_value,...,property_type,regular_person_capacity,extra_person_capacity,cost_per_extra_person,rent_price_on_regular_days,rent_price_on_special_days,rent_price_at_weekends,location_latitude,location_longitude,location_radius
0,temporary-rent,villa,karaj,mehrshahr,2024-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,6,350000.0,1500000.0,3.500000e+09,3500000.0,35.811684,50.936600,500.0
1,residential-sell,apartment-sell,tehran,gholhak,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,8.500000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0
2,residential-rent,apartment-rent,tehran,tohid,2024-10-01 00:00:00,مقطوع,26000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.703865,51.373459,NaN
3,commercial-rent,office-rent,tehran,elahiyeh,2024-06-01 00:00:00,مقطوع,95000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,residential-sell,apartment-sell,mashhad,emamreza,2024-05-01 00:00:00,NaN,NaN,NaN,مقطوع,5.750000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Changing the Values of room_count column to be understandable

In [9]:
data2['rooms_count'] = data2['rooms_count'].replace({
    'بدون اتاق': 0,
    'یک': 1,
    'دو': 2,
    'سه': 3,
    'چهار': 4,
    'پنج یا بیشتر': 5
})

C:\Users\Abtin Rahbari\AppData\Local\Temp\ipykernel_7368\2997536687.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data2['rooms_count'] = data2['rooms_count'].replace({


#### Combining floor count and apt in floors for column reduction

In [10]:
floors = data2['total_floors_count']
floors.fillna('0', inplace=True)
#floors.unique()
floors = floors.replace({
    '30+': 30,
    'unselect': 0   
})
floors = floors.astype('int16')

upf = data2['unit_per_floor']    #Units per Floor
upf.fillna('0', inplace=True)
#upf.unique()
upf = upf.replace({
    'more_than_8': 9,
    'unselect': 0   
})
upf = upf.astype('int16')
data2['total_units'] = floors * upf
data2['total_units'] = data2['total_units'].replace({
    0 : np.nan
})

In [11]:
data2 = data2.drop(columns=['unit_per_floor', 'total_floors_count'])

In [12]:
del data
del floors
del upf

In [13]:
data2.created_at_month = pd.to_datetime(data2.created_at_month)
data2.created_at_month = data2.created_at_month.dt.strftime('%Y-%m')